# 1️⃣ Install and Verify libtorrent

In [ ]:
!python -m pip install --upgrade pip setuptools wheel -q

import sys
import subprocess

def try_install():
    try:
        import libtorrent
        print(" libtorrent already available!")
        return True
    except:
        pass

    print(" Installing libtorrent...")

    try:
        subprocess.run(['apt-get', 'update', '-qq'], check=True, capture_output=True)
        subprocess.run(['apt-get', 'install', '-y', '-qq',
                        'build-essential', 'libboost-all-dev',
                        'libssl-dev', 'python3-dev'],
                       check=True, capture_output=True)
        result = subprocess.run([sys.executable, '-m', 'pip', 'install',
                                'libtorrent', '--no-binary', ':all:'],
                               capture_output=True, text=True, timeout=300)
        import libtorrent
        print(" Successfully built from source!")
        return True
    except Exception as e:
        print(f" Source build failed: {str(e)[:100]}")

    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'libtorrent', '-q'],
                       check=True, capture_output=True)
        import libtorrent
        print(" Installed prebuilt package!")
        return True
    except:
        pass

    print(" libtorrent has issues with Python 3.12")
    print(" Solution: Runtime > Change runtime type > Python 3.10")
    return False

if try_install():
    print("\n All dependencies ready!")
else:
    sys.exit(1)


# 2️⃣ Initialize Torrent Session

In [ ]:
import libtorrent as lt
import time

ses = lt.session()
ses.listen_on(6881, 6891)

settings = {
    'user_agent': 'libtorrent/2.0',
    'listen_interfaces': '0.0.0.0:6881',
    'enable_outgoing_utp': True,
    'enable_incoming_utp': True,
    'enable_outgoing_tcp': True,
    'enable_incoming_tcp': True,
}
ses.apply_settings(settings)
ses.add_dht_router("router.utorrent.com", 6881)
ses.add_dht_router("router.bittorrent.com", 6881)
ses.add_dht_router("dht.transmissionbt.com", 6881)
ses.start_dht()
downloads = []

print(" libtorrent session initialized!")

# 3️⃣ Setup Local and Google Drive Paths

In [ ]:
import os
from google.colab import drive

local_path = "/content/torrents"
os.makedirs(local_path, exist_ok=True)

drive.mount('/content/drive')
drive_path = "/content/drive/My Drive/Torrents"
os.makedirs(drive_path, exist_ok=True)

print(f" Local path: {local_path}\n Drive path: {drive_path}")

# 4️⃣ Add Magnet and List Files

In [ ]:
magnet_link = input("Paste magnet link: ").strip()

params = {
    'save_path': local_path,  # Téléchargement local
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
}

handle = lt.add_magnet_uri(ses, magnet_link, params)
downloads.append(handle)
print(" Fetching metadata...")

while not handle.has_metadata():
    time.sleep(1)

info = handle.get_torrent_info()
files = info.files()

print("\n FILES IN TORRENT:\n")
for i in range(files.num_files()):
    size_gb = files.file_size(i) / (1024**3)
    print(f"[{i}] {files.file_path(i)} — {size_gb:.2f} GB")

# 5️⃣ Select Files to Download

In [ ]:
selection = input("\n Enter file indexes to download (ex: 0,2,3): ")

indexes = [int(i.strip()) for i in selection.split(",")]

priorities = [0] * files.num_files()
for i in indexes:
    priorities[i] = 1

handle.prioritize_files(priorities)

total_size = sum(files.file_size(i) for i in indexes)
print(f"\n Selected size: {total_size / (1024**3):.2f} GB")

# 6️⃣ Download Progress Monitor

In [ ]:
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued", "checking", "downloading metadata",
    "downloading", "finished", "seeding",
    "allocating", "checking fastresume"
]

layout = widgets.Layout(width='auto')
style = {'description_width': 'initial'}
progress_bars = [widgets.FloatProgress(min=0, max=100, description='', bar_style='info', style=style, layout=layout) for _ in downloads]
labels = [widgets.Label(value='') for _ in downloads]

for i in range(len(downloads)):
    display(widgets.VBox([labels[i], progress_bars[i]]))

print("\n Starting downloads...\n")

completed_count = 0
while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        try:
            if not download.is_valid():
                continue
            bar = progress_bars[index + next_shift]
            label = labels[index + next_shift]
            s = download.status()
            if s.is_seeding or s.is_finished:
                torrent_name = s.name if s.name else "Unknown"
                bar.value = 100
                bar.bar_style = 'success'
                label.value = f" {torrent_name} - COMPLETED!"
                next_shift -= 1
                completed_count += 1
                try:
                    ses.remove_torrent(download)
                except:
                    pass
                downloads.remove(download)
                print(f"\n Download #{completed_count} complete: {torrent_name}")
                print(f" Saved to: {local_path}")
            else:
                progress = s.progress * 100
                bar.value = progress
                name = s.name if s.name else 'Fetching metadata...'
                dl_speed = s.download_rate / 1000
                ul_speed = s.upload_rate / 1000
                peers = s.num_peers
                state = state_str[s.state] if s.state < len(state_str) else 'unknown'
                label.value = f" {name}\n⬇️ {dl_speed:.1f} KB/s | ⬆️ {ul_speed:.1f} KB/s | 👥 {peers} peers | {state} | {progress:.1f}%"
                if progress < 30:
                    bar.bar_style = 'danger'
                elif progress < 70:
                    bar.bar_style = 'warning'
                else:
                    bar.bar_style = 'success'
        except Exception as e:
            print(f" Error monitoring download: {str(e)[:50]}")
            continue
    time.sleep(1)

print("\n Download(s) completed!")


# 8️⃣ Zip Torrent Folder Automatically

In [ ]:
import os
import zipfile

def get_latest_torrent_folder(base_path):
   
    folders = [
        os.path.join(base_path, d)
        for d in os.listdir(base_path)
        if os.path.isdir(os.path.join(base_path, d))
    ]

    if not folders:
        raise Exception("No torrent folder found")

    return max(folders, key=os.path.getmtime)


def zip_torrent_folder_in_parts(torrent_folder, part_size_gb=5):
   
    part_size = part_size_gb * 1024**3
    folder_name = os.path.basename(torrent_folder)
    full_zip = os.path.join(local_path, f"{folder_name}_full.zip")

    print(f"Compressing torrent folder: {torrent_folder}")

  
    with zipfile.ZipFile(full_zip, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(torrent_folder):
            for file in files:
                full_path = os.path.join(root, file)
                arcname = os.path.relpath(full_path, torrent_folder)
                zipf.write(full_path, arcname)

    print(f"Full ZIP created: {full_zip}")

    parts = []
    with open(full_zip, "rb") as f:
        part = 1
        while True:
            chunk = f.read(part_size)
            if not chunk:
                break

            part_path = os.path.join(
                local_path,
                f"{folder_name}_part{part:03d}.zip"
            )

            with open(part_path, "wb") as pf:
                pf.write(chunk)

            print(
                f"Created part {part} "
                f"({len(chunk) / (1024**3):.2f} GB)"
            )

            parts.append(part_path)
            part += 1

    return parts
torrent_folder = get_latest_torrent_folder(local_path)
print(f"Torrent folder found: {torrent_folder}")

zip_parts = zip_torrent_folder_in_parts(
    torrent_folder,
    part_size_gb=5
)

print("\nZIP parts created:")
for i, part in enumerate(zip_parts):
    print(f"[{i}] {part}")


# Upload ZIP Files to Google Drive


In [ ]:
import os
import shutil

def upload_zip_parts_to_drive(zip_parts, drive_path):
    if not os.path.isdir(drive_path):
        raise ValueError("Google Drive path not found")

    uploaded = []
    for part in zip_parts:
        if not os.path.isfile(part):
            continue

        destination = os.path.join(drive_path, os.path.basename(part))
        shutil.copy2(part, destination)
        print(f"Uploaded: {destination}")
        uploaded.append(destination)

    return uploaded
